<img src="header.png" align="left"/>

# Anwendungsbeispiel Dimensionality reduction and clustering


Das Ziel dieses Beispiels ist die Erklärung von **dimensionality reduction** und **clustering**. Dabei wird versucht die hochdimensionalen MNIST Daten auf die wesentlichen Dimensionen zu reduzieren und dann ein Clustering durchzuführen. Danach wird ein Vergleich der gefundenen Cluster mit den vorhandenen Labels durchgeführt.

Der Code für das Beispiel wurde aus [1] entlehnt, die Daten stammen von [2].

- [1] [https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b](https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b)
- [2] [http://yann.lecun.com/exdb/mnist/](http://yann.lecun.com/exdb/mnist/)




# Import der Module  

In [ ]:
#
# Importieren der Module
#
from keras.datasets import mnist

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
#
# Abdrehen von Fehlermeldungen
#
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)

In [ ]:
#
# Einstellen der Grösse von Diagrammen
#
plt.rcParams['figure.figsize'] = [16, 10]

# Laden der Daten

In [ ]:
#
# Laden der Beispieldaten (hier sehr vereinfacht) und gleichzeitige Aufteilung in Trainings- und Testdaten
#
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
#
# Anzeige der Anzahl und Form der Samples
#
print('Trainingsdaten: X=%s, y=%s' % (x_train.shape, y_train.shape))

In [ ]:
# 
# Ändere die Matrixform der Daten
#
x_train = x_train.reshape((x_train.shape[0], 784 ))

In [ ]:
# 
# Ändere Pixelwerte von 0..255 auf einen Wert zwischen 0 und 1 in Flieskommaform
#
x_train = x_train.astype('float32')
x_train = x_train / 255.0

In [ ]:
#
# Anlegen der Featurenamen
#
feat_cols = [ 'pixel'+str(i) for i in range(x_train.shape[1]) ]

In [ ]:
#
# Anlegen des Dataframe
#
df = pd.DataFrame(x_train,columns=feat_cols)

In [ ]:
#
# Anlegen der Labels
#
df['y'] = y_train
df['label'] = df['y'].apply(lambda i: str(i))

In [ ]:
#
# Check
#
print('Size of the dataframe: {}'.format(df.shape))

In [ ]:
#
# Kurzer Check
#
df.head()

In [ ]:
# 
# Anlegen einer Permutation (zufälligen Mischung) 
#
np.random.seed(42)
rndperm = np.random.permutation(df.shape[0])
print(rndperm)

In [ ]:
#
# Anzeige von Beispielen der Daten
#
for i in range(16):
    plt.subplot(4,4,1 + i)
    plt.imshow(df.loc[rndperm[i],feat_cols].values.reshape((28,28)).astype(float), cmap=plt.get_cmap('gray'))
plt.show()

# Principal component analysis

In [ ]:
#
# Anlegen der PCA Klasse
#
pca = PCA(n_components=3)

In [ ]:
#
# Trainieren der PCA Klasse
#
pca_result = pca.fit_transform(df[feat_cols].values)

In [ ]:
#
# Speichern der Resultate und Bestimmung der Varianz der Reduktion
#
df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
df['pca-three'] = pca_result[:,2]

print('Variation bestimmt durch die principal components: {}'.format(pca.explained_variance_ratio_))

# Plot der PCA Daten

In [ ]:
sns.scatterplot(
    x="pca-one", y="pca-three",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df.loc[rndperm,:],
    legend="full",
    alpha=0.2
)

In [ ]:
#
# 3D plot der gleichen PCA
#
from matplotlib.colors import ListedColormap
cmap = ListedColormap(sns.color_palette("hls", 10))

ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=df.loc[rndperm,:]["pca-one"], 
    ys=df.loc[rndperm,:]["pca-two"], 
    zs=df.loc[rndperm,:]["pca-three"], 
    c=df.loc[rndperm,:]["y"], 
    cmap=cmap, alpha=0.2
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show()

In [ ]:
#
# Reduktion der Daten auf ein Subset von 10000 Samples
# um die Rechendauer zu beschränken
#
N = 10000
df_subset = df.loc[rndperm[:N],:].copy()
data_subset = df_subset[feat_cols].values

In [ ]:
#
# Nochmals PCA auf die reduzierten Daten (für Vergleich später)
#
pca = PCA(n_components=3)
pca_result = pca.fit_transform(data_subset)

# t-SNE


Eine gute Beschreibung des Algorithmus ist hier zu finden: [https://mlexplained.com/2018/09/14/paper-dissected-visualizing-data-using-t-sne-explained/](https://mlexplained.com/2018/09/14/paper-dissected-visualizing-data-using-t-sne-explained/).

In [ ]:
#
# Reduktion mit t-sne auf 2 Dimensionen
#
tsne = TSNE(n_components=2, verbose=1, n_iter=3000, perplexity=30, early_exaggeration=12)
tsne_results = tsne.fit_transform(data_subset)

In [ ]:
df_subset['tsne-2d-one'] = tsne_results[:,0]
df_subset['tsne-2d-two'] = tsne_results[:,1]
# df_subset['tsne-2d-three'] = tsne_results[:,2]



In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df_subset,
    legend="full",
    alpha=0.3
)

In [ ]:
ax1 = plt.subplot(1, 2, 1)
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df_subset,
    legend="full",
    alpha=0.3,
    ax=ax1
)
ax2 = plt.subplot(1, 2, 2)
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df_subset,
    legend="full",
    alpha=0.3,
    ax=ax2
)

# Hinweis

Richtigerweise würde man die hochdimensionalen Daten vor t-SNE zuerst mit Hilfe von PCA in der Dimensionalität reduzieren. Damit kann zum Beispiel die Rechenzeit für die Reduktion wesentlich verkürzt werden.

# Ein Versuch zum Clustering mit DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN

# db = DBSCAN(eps=2.44, min_samples=5) # for 3D t-sne
# db = DBSCAN(eps=3.0, min_samples=5) # for 2D t-sne

db = DBSCAN(eps=3.0, min_samples=10)
db = db.fit(tsne_results)
labels_db = db.labels_
n_clusters_ = len(set(labels_db)) - (1 if -1 in labels_db else 0)
print ("Anzahl der Cluster in t-sne Daten mit DBSCAN: {}".format( n_clusters_ ) )

In [ ]:
#
# Outliers werden auf 0 gesetzt (ist das sinnvoll?)
#
labels_db[labels_db < 0] = 0

In [ ]:
#
# Ablegen der geschätzten Labels im Dataframe
#
df_subset['y_cluster'] = labels_db

In [ ]:
ax1 = plt.subplot(1, 2, 1)
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df_subset,
    legend="full",
    alpha=0.3,
    ax=ax1
)
ax2 = plt.subplot(1, 2, 2)
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y_cluster",
    palette=sns.color_palette("hls",n_clusters_),
    data=df_subset,
    legend="full",
    alpha=0.3,
    ax=ax2
)

# Ein Versuch zum Clustering mit kMeans

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=10, random_state=0)
km = km.fit(tsne_results)

In [ ]:
labels_km = km.labels_
clusters_km = km.predict(tsne_results)
n_clusters_km = len(set(labels_km)) - (1 if -1 in labels_km else 0)
print ("Anzahl der Cluster in t-sne Daten mit KMeans: {}".format( n_clusters_km ) )

In [ ]:
#
# Outliers werden auf 0 gesetzt (ist das sinnvoll?)
#
labels_km[labels_km < 0] = 0

In [ ]:
#
# Ablegen der geschätzten Labels im Dataframe
#
df_subset['y_cluster_km'] = labels_km

In [ ]:
ax1 = plt.subplot(1, 2, 1)
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df_subset,
    legend="full",
    alpha=0.3,
    ax=ax1
)
ax2 = plt.subplot(1, 2, 2)
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="y_cluster_km",
    palette=sns.color_palette("hls",n_clusters_km),
    data=df_subset,
    legend="full",
    alpha=0.3,
    ax=ax2
)